# Predicting future states with Markov Chains
We have a set of sequences followed by users in the system. We need to implement a Markov Chain representation that we can use to obtain predictions on future states. Also we want this representation to be of order $>1$.  
Let's start by loading the data and by having a look at it.

In [1]:
# load all dependencies
from support import *
# load the data
data = pd.read_csv("../data/paths_final_cass.csv", sep=";")[["element", "exit_codes", "type"]]

# let's peek into the dataframe...
data.head()

,element,exit_codes,type
0,6114|421|2814|1478|2040|3563|3622|1850|1032|20...,489|397|397|397|218|580|465|397|218|397|397|77...,0.0
1,6114|421|2814|1478|2040|3563|3622|1850|1032|20...,489|397|397|397|218|580|465|397|218|397|397|77...,2.0
2,6114|421|2814|1478|2040|3563|3622|1850|1032|20...,489|397|397|397|218|580|465|397|218|397|397|77...,2.0
3,6114|421|2814|1478|2040|3563|3622|1850|1032|20...,489|397|397|397|218|580|465|397|218|397|397|77...,2.0
4,6114|421|2814|1478|2040|3563|3622|1850|1032|20...,489|397|397|397|218|580|465|397|218|397|397|77...,2.0


### Data format
The data contain all sequence of items users touched in their session (`element`) and the exit status for each item (`exit_code`). `type` represents the goodness/badness of the sequence.  
We now need to determine the set of all possible states that we can use to build a transition matrix based on the sequences in our dataset.  
Since we are not sure all labels exist (there may be gaps in the labels), we will redefine them and normalize the counts.  

Also: we want to calculate a second order Markov Chain of the dataset so that we can correlate a pair (Element, Exit code) to the next pair (Element, Exit code).

In [2]:
# get ALL the elements (pipe separated)
all_codes = set(itertools.chain(*data['element'].apply(lambda x: str(x).split("|")).values))
print(f"{len(all_codes)} unique elements found in dataset")
# build a label map
codes_map = {item: n for n, item in enumerate(sorted(map(int, all_codes)))}
# get the new labels and joins them in a similar string
data['new_elements'] = data['element'].apply(lambda x: "|".join(str(codes_map.get(int(i), 'N/A')) for i in str(x).split("|")))

# same thing on exit states
all_codes = set(itertools.chain(*data['exit_codes'].apply(lambda x: str(x).split("|")).values))
print(f"{len(all_codes)} unique exit_codes found in dataset")
_codes_map = {item: n for n, item in enumerate(sorted(map(int, all_codes)), start=len(codes_map))}
data['new_exits'] = data['exit_codes'].apply(lambda x: "|".join(str(_codes_map.get(int(i), 'N/A')) for i in str(x).split("|")))

print(f"\n{max(_codes_map.values()) + 1} possible states single states")

288 unique elements found in dataset
97 unique exit_codes found in dataset

385 possible states single states


In [3]:
data.drop(["element", "exit_codes"], axis=1, inplace=True)
data.head()

,type,new_elements,new_exits
0,0.0,287|25|91|49|82|108|112|72|42|79|50|1|12|256|4...,371|361|361|361|325|384|366|361|325|361|361|30...
1,2.0,287|25|91|49|82|108|112|72|42|79|50|1|12|256|4...,371|361|361|361|325|384|366|361|325|361|361|30...
2,2.0,287|25|91|49|82|108|112|72|42|79|50|1|12|256|4...,371|361|361|361|325|384|366|361|325|361|361|30...
3,2.0,287|25|91|49|82|108|112|72|42|79|50|1|12|256|4...,371|361|361|361|325|384|366|361|325|361|361|30...
4,2.0,287|25|91|49|82|108|112|72|42|79|50|1|12|256|4...,371|361|361|361|325|384|366|361|325|361|361|30...


Let's now associate each element to its exit code in the sequence:

In [4]:
# this mixes and matches items and exit codes
pair_sequences = make_sequences(data)

### Implementation
Luckily for us, someone took care of the implementation of an initial version of High Order MC.  
The implementation uses sparse matrices to represent the (very) highly dimensional space that describes all possible states.

In [5]:
# install the High Order Markov chains module
# ! pip install --upgrade HOMarkov

In [6]:
# from HOMarkov import markov
import markov
number_of_states = max(_codes_map.values()) + 1 # zero based :)
mc = markov.MarkovChain(number_of_states, order=2)

In [7]:
%%time
# this will take about 30 seconds for the big dataset...
mc.fit(list(pair_sequences))

CPU times: user 217 ms, sys: 3.24 ms, total: 220 ms
Wall time: 226 ms


In [8]:
print(f"Number of non-zero transitions: {mc.transition_matrix.count_nonzero()}")
print(f"Number of total transitions: {mc.number_of_states ** 2}")

Number of non-zero transitions: 886
Number of total transitions: 148225


We can now get all non-zero elements in the transition matrix and represent them with their states:

In [9]:
# get coordinates of non-zero elements in transition matrix
non_zero_indices = list(zip(*mc.transition_matrix.nonzero()))
state_lookup = mc.possible_states_lookup()
state_transitions = [(state_lookup.get(i), state_lookup.get(j)) for i, j in non_zero_indices]
possible_start_states = set(i for i, _ in state_transitions)

In [19]:
@interact(
    initial_state=map(str, sorted(possible_start_states)),
    steps=IntSlider(1, 1, 5),
    threshold=IntSlider(0, 0, 100, 5),
    actual_weight=True
)
def draw_future(initial_state, steps, threshold, actual_weight):
    
    if not actual_weight:
        print("Using relative weight")
        
    state_index = (mc.possible_states.get(eval(initial_state)))
    state_representation = np.zeros(len(mc.possible_states))
    state_representation[state_index] = 1
    
    try:
        states = mc.evolve_states(state_representation, num_steps=steps, threshold=threshold*0.01)
    except TypeError as exc:  # no states above threshold
        print("No states above current threshold")
    else:
        pos = build_pos(states)  
        G = mc.generate_graph(states, actual=actual_weight)
        edge_labels = reset_edge_labels(G)
        labels = build_node_labels(G, state_lookup)

        nx.draw_networkx_edge_labels(G, pos, font_size=15, font_weight="bold", edge_labels=edge_labels, alpha=0.7)
        nx.draw_networkx(G, pos, width=[attr["weight"] * 5  for i, j, attr in G.edges(data=True)])
        label_offset = max([abs(i[1]) for i in pos.values()] + [0.05])
        nx.draw_networkx_labels(
            G,
            {k: (v[0], v[1] + 0.05 * label_offset) for k, v in pos.items()},
            labels,
            font_weight="bold",
            font_size=15
        )
        plt.axis("off")
        plt.show()
        
        # 51, 361
        # 58, 361
        # 65, 361
        # 76, 372
        # 364, 21
        # 216, 361

In [11]:
state_lookup[139138]

(361, 153)

In [12]:
import collections

collections.Counter(mc.transition_matrix.nonzero()[0]).most_common()

[(108931, 6),
 (139138, 6),
 (25771, 5),
 (116282, 5),
 (139183, 5),
 (687, 4),
 (34609, 4),
 (40420, 4),
 (55811, 4),
 (107006, 4),
 (132546, 4),
 (139068, 4),
 (139069, 4),
 (139144, 4),
 (142930, 4),
 (146481, 4),
 (17686, 3),
 (27696, 3),
 (41176, 3),
 (56143, 3),
 (63465, 3),
 (93146, 3),
 (96996, 3),
 (102386, 3),
 (122464, 3),
 (125301, 3),
 (138987, 3),
 (138995, 3),
 (138996, 3),
 (139164, 3),
 (139208, 3),
 (140954, 3),
 (3056, 2),
 (4999, 2),
 (6136, 2),
 (7291, 2),
 (7603, 2),
 (7676, 2),
 (8378, 2),
 (10683, 2),
 (11911, 2),
 (19611, 2),
 (21549, 2),
 (21921, 2),
 (23461, 2),
 (25386, 2),
 (26926, 2),
 (28081, 2),
 (30410, 2),
 (31895, 2),
 (32243, 2),
 (32628, 2),
 (36551, 2),
 (36936, 2),
 (37321, 2),
 (37706, 2),
 (42711, 2),
 (45775, 2),
 (56562, 2),
 (57276, 2),
 (58102, 2),
 (58844, 2),
 (60815, 2),
 (61916, 2),
 (68517, 2),
 (68914, 2),
 (69209, 2),
 (69983, 2),
 (71165, 2),
 (75400, 2),
 (75798, 2),
 (76598, 2),
 (77692, 2),
 (80056, 2),
 (82751, 2),
 (90069, 2),
 